### References

Countz, T., 2020. Calculate The Decision Boundary Of A Single Perceptron - Visualizing Linear Separability. [online] Medium. Available at: <https://medium.com/@thomascountz/calculate-the-decision-boundary-of-a-single-perceptron-visualizing-linear-separability-c4d77099ef38> [Accessed 20 October 2020].

GeeksforGeeks. 2020. Perpendicular Distance Between A Point And A Line In 2 D - Geeksforgeeks. [online] Available at: <https://www.geeksforgeeks.org/perpendicular-distance-between-a-point-and-a-line-in-2-d/?ref=lbp> [Accessed 20 October 2020].

Project 2 lab powerpoint

In [ ]:
import numpy as np
import csv
from numpy import genfromtxt
import matplotlib.pyplot as plt
import cvxopt
from cvxopt import matrix, solvers
import math
%matplotlib inline

## Importing Dataset and Plotting the points

Using the numpy function genfromtxt we can import the csv files form the computer and store them into variables X and y so that they be plotted on a singular graph. The graph shows the various indexes of data where y=1 and y=-1, and plots them with their corresponding values in X. The orange and purple dots represents y=1 and y=-1 respectively. From this we can see that the corrsponding y=-1 values are much higher than its y=1 counterparts and are also more precise.

In [ ]:
# Use numpy function genfromtxt(…, delimiter=…) to load from files.
# Store the data from “X.csv” to X and target from “y.csv” to y
X = np.genfromtxt("X.csv",delimiter=",")
y = np.genfromtxt("y.csv",delimiter=",")

# Use np.where to find all index of data which y=1 and store them to idx_1
idx_1 = np.where(y == 1)

# same as y=-1, store them to idx_2
idx_2 = np.where(y == -1)

# make the plot use plt.scatter(X[idx_1,0], X[idx_1,1], s=30, c='b', marker="o")
plt.scatter(X[idx_1,0], X[idx_1,1], s=30, c='purple', marker="o")
plt.scatter(X[idx_2,0], X[idx_2,1], s=30, c='orange', marker="o")

# Set the x label with x1 and y label with x2
plt.xlabel("x1")
plt.ylabel("x2")

# plt.show()
plt.show()

## SVM Primal  Problem

Using the CVXOPT function imported from above, the quadratic problem solver,cvxopt.solvers.qp, will solve the problem:

$$\begin{eqnarray}\left.\begin{aligned}  
&\min_{w}\frac{1}{2}||w||^{2} & \nonumber \\\
&\textrm{where} \quad w = x^{T}Px - q^{T}x \quad \\\
&\textrm{s.t.}  \quad y^{(i)}(w^{T}{\bf x}^{(i)} + w_0) \ge 1 \quad \forall i \\\
&\textrm{and}   \quad Gx \le h \quad \end{aligned}\right.\end{eqnarray}$$

The answer from this will give us the maximum margins for the SVM primal problem. The solver requires 2 manditory inputs P, Q which have to be cvxopt matrices. P is the cvxopt matix of a an identity array where the first one in the leading diagonal is now a zero and q is cvxopt matrix of a zero matrix.

Each row in X was made into an array with a leading 1 and added to a list where they were then inversed and made to form G the cvxopt matrix of the above list. h is then defined as a cvxopt matrix of array of ones where the amount is the number of rows in X.

These come together to find the linear svm of X and y by using the quadratic problem solver. The linear svm will divide the data points of X and y into 2 different classes by using a best fit line. 

In [ ]:
def linear_svm(X,y):
    solvers.options['show_progress'] = False
    
#     store the shape of X to two variables: N,F    
    N = X.shape[0]
    F = X.shape[1]
    
#     create the Identity matrix using np.diag and np.ones    
    I = np.ones([F+1, F+1]) # creates a F+1 x F+1 matrix of ones
    I = np.diag(np.diag(I)) # takes the leading diagonal from matrix above and makes it into a single diagonal matrix

#     create the Q matrix using np.zeros
    Q = np.zeros((F+1,F+1))

#     for each element in Q:
    for r in range(F+1):
        for c in range(F+1):
#         when row number is 0, set Q[row, col]=0
#         when col number is 0 set Q[row,col]=0
            if r == 0 or c == 0: 
                Q[r,c] = 0
#         else, compute Identity [row-1,col-1] and set it to Q[row,col]
            else: 
                Q[r,c] = I[r-1, c-1]

#     use cvxopt.matrix to create a new variable p with value Q
#     use cvxopt.matrix to create a new variable q with value np.zeros(F+1)
    p = cvxopt.matrix(Q)
    q = cvxopt.matrix(np.zeros(F+1))

#     create an empty list
    list=[]

#     for n in range(N):
    for n in range(N):
#     create a zero matric with size F+1
        zeros = np.zeros((F+1))
#     for each element in the matric above:
        for count in range(zeros.shape[0]): 
#             when the index=0, then set it to 1
            if count == 0: 
                zeros[count] = 1
#             else, set the value to X[n].T[i-1]
            else: 
                zeros[count] = X[n].T[count-1]
#         append the y[n]*updated matric to the empty list above (the one above the for loop
        list.append(y[n]*zeros)
    
#     change the empty list to the np array and times -1
    list = np.array(list) * -1
    
#     use cvxopt.matrix to convert above np array and store it in a variable: G
#     create a variable named h with value np.ones(N)*-1 and convert it to cvxopt
    G = cvxopt.matrix(list)
    h = np.ones(N)*-1
    h = cvxopt.matrix(h)

#     solve the primal using cvxopt.solvers.qp
#     return the answer.
    return(cvxopt.solvers.qp(p,q,G,h))

# fit svm classifier
# print the weights
sol = linear_svm(X,y)
print("Weights:",sol["x"])

### Plotting the  decision boundary

The slope and intercept of the decision boundary is found using a formula from Countz. this formula takes the bias of the weights given in the previous section and uses it combined with the others to get the gradient and the intercept. 

$$\begin{eqnarray}\left.\begin{aligned}  
&gradient = -\frac{\frac{w_0}{w_2}}{\frac{w_0}{w_1}} \end{aligned}\right.\end{eqnarray}$$

$$\begin{eqnarray}\left.\begin{aligned}  
&y-intercept = -\frac{w_0}{w_2} \end{aligned}\right.\end{eqnarray}$$

With these variables we calculated, the line can be plotted using any array of numbers for x values and then solving for their corresponding y values on the line. This willthen give us the decsion boundary at the max margin. 

In [ ]:

def plot_data_with_decision_boundary(X, y, w, w0, fig_size=(15, 9), labels=['x1', 'x2']):
#     plot the dataset
    plt.figure(figsize = fig_size)
    
    idx_1 = np.where(y == 1)
    idx_2 = np.where(y == -1)

    plt.scatter(X[idx_1,0], X[idx_1,1], s=30, c='purple', marker="o")
    plt.scatter(X[idx_2,0], X[idx_2,1], s=30, c='orange', marker="o")
    
    plt.xlabel("x1")
    plt.ylabel("x2")
    
#     find the slope of the decision boundary
    m = -(w0/w[2])/(w0/w[1])
    
#     find the intercept.
    c = -w0/w[2]

#     generate several x values np.arrange()
    x = np.arange(0,5)

#     calculate its y values using intercept and slope
    Y = m*x + c
    
#     plot a line
    plt.plot(x,Y)
    
plot_data_with_decision_boundary(X,y,sol["x"],sol["x"][0],(15,9),labels=["x1","x2"])

### Closest points to decision boundary

Using a known formula to calculate the perpendicula distancebetween eveyr point in X and the decision boundary,exempt from negative values, we can get a list of the closest possible points in the graph. these points can than be used to calculate the margin. 

                                 Distance = (| a*x1 + b*y1 + c |) / (sqrt( a*a + b*b))

The pointswere determined by how close they were to the minimum distance rounded up to the third decimal place as all the other values afterwards were arbituary as the difference was very small. This gave us a set of 3 values that had the smallest perpendicualar distance from the decision boundary. 

In [ ]:
# calculate distance from each point to the decision boundary
def distance(x, w, w0):
    
# find the slope and intercept of the decision boundary 
    m = -(w0/w[2])/(w0/w[1])
    c = -w0/w[2]
    
#compute the perpendicular distance using the formula  
    d = abs((-m * x[0] + 1 * x[1] - c)) / (math.sqrt(-m * -m + 1)) 
    
    return (d)

closepoints = []
min = 10

# find the nearest data points and its index.
for i in range(X.shape[0]):
    d = (distance(X[i],sol["x"],sol["x"][0]))
    rd = round(d, 3)# rounding the distance to the third decimal place
    if rd < min:
        min = rd
        closepoints.clear()#clearing the list of a new minimal is found so that only indexes present are of those very close to the new minimal
        closepoints.append([d,i])
    elif rd == min:
        closepoints.append([d,i])#adds the index of any point that is closest to the boundary

print("The points closest to the decision boundary:")
for a in range(len(closepoints)):
    print("Point: ", X[a],",Index: ", closepoints[a][1],", Distance: ",closepoints[a][0])

In [ ]:
def f_primal(x): 
#     return the predicted value using svm primal
    w =[]
    w.append(sol["x"][1])
    w.append(sol["x"][2])
    
    Y =np.dot(x,w) + w[1]
    
    if Y > 0:
      return 1
    else:
      return -1
# using f_ primal() to predict (3.0, 1.5) and (1.2, 3.0) and plot the figure.

f1 = f_primal((3.0,1.5))
f2 = f_primal((1.2,3.0))
